In [4]:
import brkraw as brk
from typing import cast
from pprint import pprint
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

def reorient_to_ras(data, affine):
    ornt = nib.orientations.io_orientation(affine)
    ras_ornt = np.array([[0, 1], [1, 1], [2, 1]])  # RAS
    transform = nib.orientations.ornt_transform(ornt, ras_ornt)
    new_data = nib.orientations.apply_orientation(data, transform)
    new_affine = affine @ nib.orientations.inv_ornt_aff(transform, data.shape)
    return new_data, new_affine

def _apply_affine(aff, ijk):
    ijk = np.asarray(ijk, dtype=float)
    ijk1 = np.c_[ijk, np.ones(len(ijk))]
    xyz = ijk1 @ aff.T
    return xyz[:, :3]

def _extent_for_plane(shape, affine, plane, fixed_index):
    nx, ny, nz = shape
    if plane == "axial":  # x-y at k
        corners = [(0, 0, fixed_index),
                   (nx - 1, 0, fixed_index),
                   (0, ny - 1, fixed_index),
                   (nx - 1, ny - 1, fixed_index)]
        world = _apply_affine(affine, corners)
        xs, ys = world[:, 0], world[:, 1]
        return [xs.min(), xs.max(), ys.min(), ys.max()]
    if plane == "coronal":  # x-z at j
        corners = [(0, fixed_index, 0),
                   (nx - 1, fixed_index, 0),
                   (0, fixed_index, nz - 1),
                   (nx - 1, fixed_index, nz - 1)]
        world = _apply_affine(affine, corners)
        xs, zs = world[:, 0], world[:, 2]
        return [xs.min(), xs.max(), zs.min(), zs.max()]
    if plane == "sagittal":  # y-z at i
        corners = [(fixed_index, 0, 0),
                   (fixed_index, ny - 1, 0),
                   (fixed_index, 0, nz - 1),
                   (fixed_index, ny - 1, nz - 1)]
        world = _apply_affine(affine, corners)
        ys, zs = world[:, 1], world[:, 2]
        return [ys.min(), ys.max(), zs.min(), zs.max()]
    raise ValueError("plane must be axial/coronal/sagittal")

def plot_orthogonal_world(dataobj, affine, title):
    nx, ny, nz = dataobj.shape
    i, j, k = nx // 2, ny // 2, nz // 2

    views = [
        ("Axial (x, y)", dataobj[:, :, k].T, _extent_for_plane(dataobj.shape, affine, "axial", k)),
        ("Coronal (x, z)", dataobj[:, j, :].T, _extent_for_plane(dataobj.shape, affine, "coronal", j)),
        ("Sagittal (y, z)", dataobj[i, :, :].T, _extent_for_plane(dataobj.shape, affine, "sagittal", i)),
    ]

    fig, axes = plt.subplots(1, 3, figsize=(12, 4), dpi=70)
    for ax, (label, img, extent) in zip(axes, views):
        ax.imshow(img, cmap="gray", origin="lower", extent=extent)
        ax.set_title(label)
        ax.set_aspect("equal")
        ax.axis("off")

    fig.suptitle(title)
    plt.tight_layout()

In [5]:
raw = brk.load('./data/TestSliceReadoutOrient.zip')
scan_info = cast(dict, raw.info(scope='scan', show_reco=False, as_dict=True))
orient_info = {}
for scan_id, info in scan_info.items():
    orient_info[scan_id] = (info["SliceOrient"], info["ReadOrient"])
pprint(orient_info)

{5: (['coronal'], ['H_F']),
 7: (['coronal'], ['L_R']),
 8: (['axial'], ['L_R']),
 9: (['axial'], ['A_P']),
 10: (['sagittal'], ['A_P']),
 11: (['sagittal'], ['H_F'])}
